In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

from importlib import reload

from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing 


from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'



In [4]:
from predicament.utils.file_utils import load_dataframe_and_config
import predicament.utils.config_parser
reload(predicament.utils.config_parser)
from predicament.utils.config_parser import config_to_dict

from predicament.utils.config import FEATURED_BASE_PATH
from predicament.data.features import IDEAL_FEATURE_GROUP

from predicament.evaluation.balancing import get_group_label_counts
from predicament.evaluation.balancing import balance_data
from predicament.evaluation.grouping import get_group_assignments
from predicament.evaluation.staging import get_design_matrix
from predicament.evaluation.results import output_model_best_from_results
from predicament.evaluation.results import save_results_df_to_file

from predicament.evaluation.hyperparameters import get_param_scopes
from predicament.evaluation.hyperparameters import get_param_search_object

from predicament.models.mlp_wrappers import ThreeHiddenLayerClassifier


In [5]:
# high level choices
subdir = 'binary_dreem_4secs' # dataset
held_out = 'participant'
is_balanced = True # balance data set
use_only_ideal_features = True # restrict to preferred ideal features
standardise_data = False
max_iter_opt = 200
n_iter = 50 # number of iterations for your search
new_search = True # restarts the search
random_state = 43
use_callback = True

## Load featured data and balance if required

Before running this, you will need to generate featured data. See README file for details. For the variable `subdir` above replace this with the subdirectory name of the featured data.

In [6]:
featured_data_dir = os.path.join(FEATURED_BASE_PATH,subdir)

featured_df, featured_config = load_dataframe_and_config(
    featured_data_dir, 'featured.csv')



In [7]:
n_channels = featured_config['LOAD']['n_channels']
data_format = featured_config['LOAD']['data_format']
channels = featured_config['LOAD']['channels']
participant_list = featured_config['LOAD']['participant_list']
sample_rate = featured_config['LOAD']['sample_rate']
Fs = sample_rate
window_size = featured_config['WINDOWED']['window_size']
time = window_size/sample_rate
print(f"sample_rate: {sample_rate}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")


sample_rate: 250, n_samples = 1024, time: 4.096s, n_channels: 4


In [8]:
subject_condition_counts = get_group_label_counts(featured_df, 'participant', 'condition')
if is_balanced:
    print(f"before balancing: subject_condition_counts = {subject_condition_counts}")
    # balance featured data
    featured_df = balance_data(featured_df, group_col='participant', label_col='condition')
    subject_condition_counts = get_group_label_counts(featured_df, 'participant', 'condition')

    print(f"after balancing: subject_condition_counts = {subject_condition_counts}")

print(f"np.sum(subject_condition_counts, axis=0) {np.sum(subject_condition_counts, axis=0)}")
print(f"np.sum(subject_condition_counts, axis=1) {np.sum(subject_condition_counts, axis=1)}")
print(f"np.sum(subject_condition_counts) {np.sum(subject_condition_counts)}")
    


before balancing: subject_condition_counts = [[2657.    0.]
 [2638.    0.]
 [2072.  110.]
 [2423.  805.]
 [2423.  344.]
 [1501.  110.]
 [2122.  461.]
 [2282.  688.]
 [2079.  110.]
 [2027.  285.]
 [2540.  344.]
 [2540.  805.]]
after balancing: subject_condition_counts = [[339.   0.]
 [339.   0.]
 [339. 110.]
 [339. 805.]
 [339. 344.]
 [339. 110.]
 [339. 461.]
 [339. 688.]
 [339. 110.]
 [339. 285.]
 [339. 344.]
 [339. 805.]]
np.sum(subject_condition_counts, axis=0) [4068. 4062.]
np.sum(subject_condition_counts, axis=1) [ 339.  339.  449. 1144.  683.  449.  800. 1027.  449.  624.  683. 1144.]
np.sum(subject_condition_counts) 8130.0


## Define model and hyperparamer search

In [9]:
overrides = dict()
excludes = list()
# the base model to tune
# estimator = SVC()
#estimator = GradientBoostingClassifier()
# estimator = RandomForestClassifier()
# max_iter_opt = 1
# estimator = MLPClassifier(max_iter=max_iter_opt)
estimator = ThreeHiddenLayerClassifier()
# excludes = ['layer3'] # for 2 (hidden) layer MLP (leave empty for 3 layer MLP)
# excludes = ['layer2', 'layer3'] # for 1 (hidden) layer MLP
print(f"estimator = {estimator}")

# search_type = 'random_search'
search_type = 'bayesian_optimization'

# now create the parameter search object and run the hyperparameter search
param_scopes = get_param_scopes(
    search_type, estimator, excludes=excludes, **overrides)
print(f"param_scopes = {param_scopes}")


estimator = None
param_scopes = {'layer1': Integer(low=10, high=400, prior='uniform', transform='identity'), 'layer2': Integer(low=10, high=200, prior='uniform', transform='identity'), 'layer3': Integer(low=10, high=100, prior='uniform', transform='identity'), 'activation': Categorical(categories=('tanh', 'relu'), prior=None), 'solver': Categorical(categories=('sgd', 'adam'), prior=None), 'alpha': Real(low=1e-06, high=0.01, prior='log-uniform', transform='identity'), 'learning_rate': Categorical(categories=('constant', 'adaptive'), prior=None), 'learning_rate_init': Real(low=1e-06, high=1.0, prior='log-uniform', transform='identity')}


## Define data properties and data-split 

In [10]:
feature_set = featured_config['FEATURED']['feature_set']
if use_only_ideal_features:
    feature_set = list(IDEAL_FEATURE_GROUP.intersection(feature_set))
    
print(f"feature_set = {feature_set}")

# extract input data
# use all features in file
feature_types, feature_names, designmtx = get_design_matrix(
    featured_df, feature_set)
# extract labels
labels = featured_df['condition'].values.astype(int)

if standardise_data:
    scaler = preprocessing.StandardScaler().fit(designmtx)
    designmtx = scaler.transform(designmtx)
    
# prepare Hold one group out cross validation
held_out, groups, group_assignments = get_group_assignments(featured_df)
n_groups = len(groups)
# cross validation splits    
group_kfold = GroupKFold(n_splits=n_groups)

feature_set = ['SampleEntropy', 'MeanFreq', 'MaxFreqInd', 'IQR', 'FreqKurtosis', 'LyapunovExponent', 'MAD', 'Min', 'arCoeff', 'LempelZivComplexity', 'Mean', 'Correlation', 'Max', 'Hurst', 'SD']


## Define and Execute hyperparameter search strategy 

In [11]:
# fix to avoid error in BayesSearchCV.fit
import numpy as np
np.int = int

def create_callback_and_storage(param_search):
    intermediate_results = []
    # Define a custom callback function to store intermediate results
    def on_step(optim_result):
        # Store the current state of the optimization process
        intermediate_results.append((optim_result.func_vals[-1], optim_result.x_iters[-1]))

        # Print out the best score and best parameters found so far
        best_score = -optim_result.fun
        print("Best score: %s" % best_score)
        print("Best parameters: %s" % optim_result.x)
    return on_step, intermediate_results


param_search  = get_param_search_object(
    search_type, estimator, param_scopes=param_scopes, 
    n_iter = n_iter, cv=group_kfold,
    verbose=2, random_state=random_state, n_jobs=-1,
    fit_params={'X': designmtx, 'y': labels, 'callbacks': None},
    refit=False  # Ensure that the search does not refit the model with the best parameters found so far
)

if use_callback:
    if new_search:
        on_step, intermediate_results = create_callback_and_storage(param_search)
    else:
        param_search.fit_params['search_results'] = intermediate_results
    try:
        # Fit the random search model
        _ = param_search.fit(X=designmtx, y=labels, groups=group_assignments, callback=on_step)
    except Exception as e:
        print("Exception occurred:", str(e))
    finally:
        # Print or process intermediate results even if an error occurs
        print("Intermediate Results:")
        for i, (scores, params) in enumerate(intermediate_results):
            print(f"Iteration {i + 1}: Scores - {scores}, Params - {params}")
        new_search = False
        random_state = np.random.randint(100000)
else:
    _ = param_search.fit(X=designmtx, y=labels, groups=group_assignments)


Fitting 12 folds for each of 1 candidates, totalling 12 fits


/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Best score: 0.29516729595376373
Best parameters: ['relu', 1.0310736334774332e-05, 98, 91, 90, 'adaptive', 0.03343948246902571, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.46778507171499734
Best parameters: ['tanh', 0.005885338806523476, 188, 117, 84, 'constant', 0.000435124446418149, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.46778507171499734
Best parameters: ['tanh', 0.005885338806523476, 188, 117, 84, 'constant', 0.000435124446418149, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.46778507171499734
Best parameters: ['tanh', 0.005885338806523476, 188, 117, 84, 'constant', 0.000435124446418149, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.46778507171499734
Best parameters: ['tanh', 0.005885338806523476, 188, 117, 84, 'constant', 0.000435124446418149, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.4677850

/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/n

Best score: 0.47120828392148634
Best parameters: ['relu', 0.00034973710968534325, 16, 158, 77, 'constant', 2.403993417866231e-06, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.47120828392148634
Best parameters: ['relu', 0.00034973710968534325, 16, 158, 77, 'constant', 2.403993417866231e-06, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits


...

[CV] END activation=relu, alpha=1.0310736334774332e-05, layer1=98, layer2=91, layer3=90, learning_rate=adaptive, learning_rate_init=0.03343948246902571, solver=sgd; total time=  11.7s
[CV] END activation=relu, alpha=1.0310736334774332e-05, layer1=98, layer2=91, layer3=90, learning_rate=adaptive, learning_rate_init=0.03343948246902571, solver=sgd; total time=  12.1s
[CV] END activation=relu, alpha=1.0310736334774332e-05, layer1=98, layer2=91, layer3=90, learning_rate=adaptive, learning_rate_init=0.03343948246902571, solver=sgd; total time=  20.6s
[CV] END activation=tanh, alpha=0.005885338806523476, layer1=188, layer2=117, layer3=84, learning_rate=constant, learning_rate_init=0.000435124446418149, solver=sgd; total time=  15.1s
[CV] END activation=tanh, alpha=0.005885338806523476, layer1=188, layer2=117, layer3=84, learning_rate=constant, learning_rate_init=0.000435124446418149, solver=sgd; total time=  18.0s
[CV] END activation=tanh, alpha=0.005885338806523476, layer1=188, layer2=117, 

Fitting 12 folds for each of 1 candidates, totalling 12 fits
[CV] END activation=relu, alpha=1.0310736334774332e-05, layer1=98, layer2=91, layer3=90, learning_rate=adaptive, learning_rate_init=0.03343948246902571, solver=sgd; total time=  12.0s
[CV] END activation=relu, alpha=1.0310736334774332e-05, layer1=98, layer2=91, layer3=90, learning_rate=adaptive, learning_rate_init=0.03343948246902571, solver=sgd; total time=  12.0s
[CV] END activation=relu, alpha=1.0310736334774332e-05, layer1=98, layer2=91, layer3=90, learning_rate=adaptive, learning_rate_init=0.03343948246902571, solver=sgd; total time=  13.1s
[CV] END activation=tanh, alpha=0.005885338806523476, layer1=188, layer2=117, layer3=84, learning_rate=constant, learning_rate_init=0.000435124446418149, solver=sgd; total time=  16.8s
[CV] END activation=tanh, alpha=0.005885338806523476, layer1=188, layer2=117, layer3=84, learning_rate=constant, learning_rate_init=0.000435124446418149, solver=sgd; total time=  10.9s
[CV] END activati

/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV] END activation=tanh, alpha=0.002998962719489213, layer1=53, layer2=200, layer3=94, learning_rate=constant, learning_rate_init=0.001718962440700763, solver=adam; total time=  26.3s
[CV] END activation=relu, alpha=2.1948616002843934e-06, layer1=10, layer2=175, layer3=100, learning_rate=adaptive, learning_rate_init=1.0, solver=sgd; total time=  31.9s
[CV] END activation=tanh, alpha=0.002998962719489213, layer1=53, layer2=200, layer3=94, learning_rate=constant, learning_rate_init=0.001718962440700763, solver=adam; total time=  20.2s
[CV] END activation=tanh, alpha=0.002998962719489213, layer1=53, layer2=200, layer3=94, learning_rate=constant, learning_rate_init=0.001718962440700763, solver=adam; total time=  35.8s
Intermediate Results:
Iteration 1: Scores - -0.29516729595376373, Params - ['relu', 1.0310736334774332e-05, 98, 91, 90, 'adaptive', 0.03343948246902571, 'sgd']
Iteration 2: Scores - -0.46778507171499734, Params - ['tanh', 0.005885338806523476, 188, 117, 84, 'constant', 0.000

KeyboardInterrupt: 

In [12]:
intermediate_results

[(-0.29516729595376373,
  ['relu',
   1.0310736334774332e-05,
   98,
   91,
   90,
   'adaptive',
   0.03343948246902571,
   'sgd']),
 (-0.46778507171499734,
  ['tanh',
   0.005885338806523476,
   188,
   117,
   84,
   'constant',
   0.000435124446418149,
   'sgd']),
 (-0.4333928841596461,
  ['tanh',
   0.00012282687710642398,
   61,
   164,
   96,
   'adaptive',
   0.9081746266547289,
   'adam']),
 (-0.4627991183374629,
  ['tanh',
   8.936224314764104e-06,
   195,
   156,
   72,
   'constant',
   0.0028898564761782627,
   'sgd']),
 (-0.4292526039413767,
  ['tanh',
   0.00014688992790007224,
   206,
   81,
   89,
   'constant',
   0.04345162765499119,
   'sgd']),
 (-0.29484936151673596,
  ['tanh',
   4.468330777342982e-05,
   175,
   53,
   93,
   'adaptive',
   0.16564418930348992,
   'sgd']),
 (-0.4689231503070009,
  ['tanh',
   2.442583506253724e-06,
   284,
   169,
   37,
   'constant',
   0.9422526961882033,
   'sgd']),
 (-0.47120828392148634,
  ['relu',
   0.00034973710968534325

In [ ]:
intermediate_results[1][0]

In [ ]:
param_search.get_params()

## Saving and outputing results

In [ ]:
result_df = pd.DataFrame(param_search.cv_results_)
i = 0
result_df.insert(i, 'model', str(estimator))
i +=1
result_df.insert(i, 'data format', data_format)
i +=1
result_df.insert(i, 'held out', held_out)
i +=1
result_df.insert(i, 'balanced', is_balanced)
i +=1
result_df.insert(i, 'n_splits', param_search.get_params()['cv'].get_n_splits())
i +=1
result_df.insert(i, 'feature set', str(feature_types))
i +=1
result_df.insert(i, 'window size', window_size)
display(result_df)
results_fname = f'{search_type}_{str(estimator)}'
print(f"Saving to {results_fname}")
save_results_df_to_file(result_df, results_fname)

In [ ]:
_ = output_model_best_from_results(result_df)


In [ ]:
label_cols = featured_config['WINDOWED']['label_cols']
feature_types = list(feature_set)
feature_types.sort()
print(f"# Feature Set:\n{feature_types}")
derived_feature_names = []
derived_feature_types = set([])
for f in featured_df.columns:
    if f in label_cols:
        continue
    elif (f[-1] == ']'):
        if (f[:-1].rstrip('0123456789')[-1] == '['):
            f = f[:-1].rstrip('0123456789')[:-1]
    else:
        f = f.rstrip('0123456789')
    for type_ in feature_set:
        if f.startswith(type_):
            derived_feature_types.add(f)
            break
derived_feature_types = list(derived_feature_types)
derived_feature_types.sort()
output = ';'.join(derived_feature_types)
print(f"Derived Feature Types:\n{output}")

In [ ]:
thresholds = np.linspace(0,0.7,51)
N = len(result_df)
props = np.empty(thresholds.size)
for t, thresh in enumerate(thresholds):
    count = np.sum(result_df['mean_test_score'] > thresh)
    props[t] = count/N
plt.plot(thresholds, props)
plt.xlabel("mean test score")
plt.ylabel("proportion greater than")

In [ ]:
result_df.columns

In [ ]:
display_thresh = 0.6
display_cols = [col for col in result_df.columns if col.startswith('param') or (col =='mean_test_score') or (col == 'std_test_score')]
result_df[result_df['mean_test_score'] >= display_thresh][display_cols]

In [ ]:
# or just show all rows
result_df[display_cols]

## Manual


In [ ]:
raise ValueError("Not typically to run")

In [ ]:
estimator = estimator
scores = np.array([-0.2918256, -0.48112939, -0.58945741, -0.52456557, -0.42854816, -0.29049613, -0.57788722, -0.46617301, -0.29712623, -0.51388248, -0.52524064, -0.48868561, -0.52410018, -0.38288302, -0.47821039, -0.59778406, -0.6001515, -0.44612275, -0.579513, -0.55742102, -0.49297139, -0.50886454, -0.57720925, -0.46039473, -0.57381827, -0.46704014, -0.46283225, -0.48787714, -0.59009014, -0.37365682, -0.41911328, -0.42111614, -0.52076175, -0.41058924, -0.53481041, -0.63543698, -0.44255759, -0.46280623])
params = [['relu', 1.0310736334774332e-05, 98, 91, 90, 'adaptive', 0.03343948246902571, 'sgd'], ['tanh', 0.005885338806523476, 188, 117, 84, 'constant', 0.000435124446418149, 'sgd'], ['tanh', 0.00012282687710642398, 61, 164, 96, 'adaptive', 0.9081746266547289, 'adam'], ['tanh', 8.936224314764104e-06, 195, 156, 72, 'constant', 0.0028898564761782627, 'sgd'], ['tanh', 0.00014688992790007224, 206, 81, 89, 'constant', 0.04345162765499119, 'sgd'], ['tanh', 4.468330777342982e-05, 175, 53, 93, 'adaptive', 0.16564418930348992, 'sgd'], ['tanh', 2.442583506253724e-06, 284, 169, 37, 'constant', 0.9422526961882033, 'sgd'], ['relu', 0.00034973710968534325, 16, 158, 77, 'constant', 2.403993417866231e-06, 'sgd'], ['relu', 3.487488243540924e-06, 31, 23, 39, 'constant', 0.050271726957264944, 'sgd'], ['relu', 0.0007356479814621087, 285, 169, 63, 'constant', 1.6685224951216229e-06, 'adam'], ['relu', 1e-06, 10, 10, 10, 'adaptive', 1e-06, 'adam'], ['tanh', 0.01, 400, 200, 10, 'constant', 1.0, 'sgd'], ['tanh', 1e-06, 400, 200, 44, 'constant', 0.001349768202312769, 'sgd'], ['tanh', 0.01, 10, 143, 99, 'constant', 0.3393618590409573, 'adam'], ['relu', 2.7269440817022083e-06, 292, 142, 96, 'adaptive', 0.00029767246344318586, 'adam'], ['relu', 0.01, 10, 107, 74, 'adaptive', 1e-06, 'adam'], ['relu', 0.01, 10, 145, 10, 'adaptive', 1e-06, 'adam'], ['tanh', 0.01, 208, 200, 10, 'adaptive', 1e-06, 'sgd'], ['tanh', 1e-06, 10, 131, 30, 'adaptive', 0.2574530911172071, 'adam'], ['relu', 0.01, 59, 127, 10, 'adaptive', 6.0072145004243325e-06, 'adam'], ['relu', 0.01, 10, 200, 100, 'adaptive', 1e-06, 'adam'], ['relu', 3.539604052351023e-05, 400, 10, 31, 'constant', 6.236342480076656e-06, 'adam'], ['tanh', 0.0009736247674544277, 400, 10, 81, 'adaptive', 0.020483453701305686, 'adam'], ['tanh', 0.01, 267, 10, 100, 'adaptive', 0.6412852324328189, 'adam'], ['tanh', 6.335749721848321e-06, 257, 200, 81, 'constant', 1e-06, 'sgd'], ['tanh', 3.492454774824146e-06, 400, 200, 41, 'constant', 8.967762690452768e-06, 'adam'], ['relu', 4.427899330633478e-06, 400, 69, 69, 'adaptive', 7.721561926202197e-06, 'adam'], ['tanh', 1e-06, 10, 90, 100, 'adaptive', 1.0, 'adam'], ['relu', 2.056570162260457e-06, 329, 152, 48, 'constant', 1e-06, 'sgd'], ['tanh', 1e-06, 10, 128, 100, 'adaptive', 1e-06, 'adam'], ['tanh', 7.691029629621723e-06, 167, 173, 48, 'adaptive', 0.4139641395339593, 'adam'], ['relu', 0.0025205170644200395, 10, 94, 43, 'adaptive', 1e-06, 'adam'], ['tanh', 3.233599384745725e-06, 301, 171, 53, 'constant', 4.627120593115604e-06, 'sgd'], ['relu', 0.00021002377160631356, 314, 127, 45, 'adaptive', 0.48143624690940207, 'adam'], ['relu', 0.01, 241, 128, 72, 'constant', 1e-06, 'adam'], ['tanh', 0.00020537419434045332, 143, 126, 93, 'adaptive', 0.3664322003663956, 'adam'], ['tanh', 0.0002641638915759486, 192, 105, 91, 'adaptive', 0.2151921166888021, 'adam'], ['relu', 0.01, 121, 10, 82, 'constant', 5.7807353220563425e-05, 'adam']]

best_index = np.argmin(scores)
print(f"best_index = {best_index}")
print(f"best score = {-scores[best_index]}")
print(f"best params = {params[best_index]}")